# Parallel Logistic Regression Classifier on Spark

Author: **Giorgio Polla**  
Date: **TODO/11/2019**  

Implementation of a Logistic Regression classifier on Spark, with Python.  
The algorithm is tested using cross-validation on the _Spam_ dataset available at https://web.stanford.edu/~hastie/ElemStatLearn/data.html.


### Libraries and constants

Import of the following libraries:  
* `pyspark` to utilize *Spark*;  
* `time` to track the time performance of the implementation;  
* `numpy` to easily operate with number arrays;  
* `random` to provide the random functionalities used in the cross-validation dataset split;  
* `matplotlib` to plot graphs;  
* `sklearn` to easily compute some evaluation metrics.  

Moreover, some operative constants used in the following are defined, and finally the Spark context is intialised.

In [1]:
import pyspark
import time
import numpy as np
import random
import matplotlib.pyplot as plt
from random import randrange
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

FILE_PATH = '../data/spam.txt'
N_WORKERS = 8
EPSILON = 1e-10

In [2]:
sc = pyspark.SparkContext()

### File reading
The spam dataset is loaded as an RDD, using the `textFile()` function.  
The RDD is then manipulated in order to have each record as a tuple **(X, y)** , where:  
**X** is an array containing the features of the example (57 float numbers);  
**y** is an integer containing the label of the example (0 or 1).  

The result is then quickly tested.

In [3]:
def read_file(file_path):
    rdd = sc.textFile(file_path)
    
    rdd = rdd.map(
        lambda x: (
            [float(el) for el in x[:-1].split()],
            int(x[-1])
        )
    )
    
    return rdd

In [15]:
rdd = read_file(FILE_PATH)
rdd.take(1)[0][0][56] # test -> 278.0

278.0

### Standardisation
Each row of the RDD is standardised; this is done by first obtaining the mean for each column, then obtaining the standard deviation for them, and finally applying the standardisation formula.  

The result is then again quickly tested.

In [18]:
def standardise(rdd):    
    n_rows = rdd.count()
    col_sum = rdd.map(
        lambda x:
            x[0]
    ).reduce(
        lambda x, y:
            [sum(el) for el in zip(x, y)]
    )
    mean = np.divide(col_sum, n_rows)
    
    variance = rdd.map(
        lambda x: np.square(x[0] - mean)
    ).reduce(
        lambda x, y:
            [sum(el) for el in zip(x, y)]
    )
    std_dev = np.sqrt(np.divide(variance, n_rows))
    
    rdd = rdd.map(
        lambda x: (
            np.divide((x[0] - mean), std_dev),
            x[1]
        )
    )
    
    return rdd

In [19]:
rdd = standardise(rdd)
rdd.take(1)[0][0][56] # test -> -0.008724

-0.00872413388250125

### Training
The training phase follows the typical Logistc Regression fashion, with some adjustments to exploit the Spark parallelisation.  
It is worth noting that the notation is consistent, and in particular:  
* **rdd** is an RDD containing the records on which to train the model;
* **x** is a data point feature array (57 float numbers);  
* **y** is a data point label (integer, either 0 or 1);  
* **b** is the regression bias term (float number);  
* **w** is the regression weights array (57 float numbers);  
* **p** is the predicted class probability for of data point (float number between 0 and 1);

In the first section there are the support functions for the proper training phase.  
* `sigmoid()` simply represents the Sigmoid function;  
* `predict_probability()` calculates and returns the predicted class probability **y** for each record, exploting the `sigmoid()` function;  
* `cross_entropy()` is the Cross Entropy function, utilised to obtain the training error of the model;  
* `gradient()` calculates the gradient for the bias term (**gradient_b**),  the gradient for the weights, in a single array (**gradient_w**), and the sum of the cross entropy error for each record.

In [7]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def predict_probability(x, b, w):
    z = np.dot(x, w) + b
    return sigmoid(z)


def cross_entropy(p, y):    
    if y == 1:
        error_loss = -np.log(p + EPSILON)
    else:
        error_loss = -np.log(1 - p + EPSILON)
    
    return error_loss


def gradient(rdd, b, w):
    gradient_b, gradient_w, error_loss = rdd.map(
        lambda x: (
            x[0],
            x[1],
            predict_probability(x[0], b, w),
        )
    ).map(
        lambda x: (
            x[2] - x[1],
            np.multiply(
                x[2] - x[1],
                x[0]
            ),
            cross_entropy(
                x[2],
                x[1]
            )
        )
    ).reduce(
        lambda x, y: (
            x[0] + y[0],
            [sum(el) for el in zip(x[1], y[1])],
            x[2] + y[2]
        )
    )
    
    return gradient_b, gradient_w, error_loss

The train function considerds the additional parameters:  
* **iterations**, the number of iterations to repeat in the training phase;
* **learning_rate**, the learing rate in the training phase;
* **lambda_reg**, the regularization parameter lambda, used to control the Ridge-like regularization adopted;
* **verbose**, controlling whether or not to stamp additional information.  

The training phase proceeds inside the main iteration loop by calculating each time the gradient for the bias **b** and the weights **w**, applying the gradient descent technique with appropriate regularization, and recording the total loss (cross-entropy loss **err_loss** plus regularization loss **reg_loss**).  
At the end of the training phase the function returns the trained bias and weights, as well as the array **loss_history** containing the records of the total loss during the whole process.

In [8]:
def train(rdd, iterations=None, learning_rate=None, 
          lambda_reg=None, verbose=None):
    n_rows = rdd.count()
    n_features = len(rdd.first()[0])
    b = 0
    w = np.zeros(n_features)
    
    if iterations == None:
        iterations = 10
    if learning_rate == None:
        learning_rate = 10
    if lambda_reg == None:
        lambda_reg = 1
    if verbose == None:
        verbose = True
    
    loss_history = []
    start_time = time.time()
    
    for it in range(iterations):
        gradient_b, gradient_w, err_loss = gradient(rdd, b, w)
        
        reg_loss = lambda_reg * np.sum(np.square(w)) / 2
        regularization = lambda_reg * w
        
        b -= learning_rate * gradient_b / n_rows
        w -= learning_rate * (gradient_w + regularization) / n_rows
        
        total_loss = (err_loss + reg_loss) / n_rows
        loss_history.append(total_loss)
        
        if verbose and it % (iterations / 5) == 0:
            print("It. %4d\t|\tLoss: %0.4f\t|\tTime: %0.2f s" %  
                  (it, total_loss, (time.time() - start_time)))
    if verbose:
        total_time = time.time() - start_time
        print('\nTotal time: %0.2f s\nIters frequency: %0.2f Hz' % 
              (total_time, iterations / total_time))
    
    return b, w, loss_history

### Performance Evaluation
The evaluation is entirely managed by the follwing two functions, following the usual parameter naming conventions.  
* `classify()` is a support function that returns wether a data point has been classified as a _True Positve_ (**tp**), a _True Negative_ (**tn**), a _False Negative_ (**fn**) or a _False Positve_ (**fp**);  
* `evaluate()` computes the traditional evaluation metric for classification, and in particular the **Accuracy**, **Precision**, **Recall**, **Specificity**, **F1-Score**, final **Cost Error**, area under the ROC curve (**AUC**) and precision-recall score (**P-R Score**), and returns them packed in a dictionary. If the parameter **plot** is positively flagged, the two plots of the ROC and precision-recall curves are displayed.

In [9]:
def classify(p, y, threshold=0.5):
    if (p >= threshold) == y:
        return 'tp' if y==1 else 'tn'
    else:
        return 'fn' if y==1 else 'fp'
            
        
def evaluate(rdd, b, w, loss_history=[None], plot=True):
    
    rdd_pred = rdd.map(
        lambda x: (
            predict_probability(x[0], b, w),
            x[1]
        )
    )
    
    predictions = rdd_pred.map(
        lambda x:
            x[0]
    ).collect()
    
    corrects = rdd_pred.map(
        lambda x:
            x[1]
    ).collect()
    
    c_m = rdd_pred.map(
        lambda x: (
            classify(x[0], x[1]),
            1
        )
    ).reduceByKey(
        lambda x, y:
            x+y
    )
    c_m = dict(c_m.collect())
    
    roc1, roc2, _ = roc_curve(corrects, predictions)
    
    results = {}
    
    results['Confusion Matrix'] = c_m
    
    results['Accuracy'] = (c_m['tp'] + c_m['tn']) \
        / (c_m['tp'] + c_m['tn'] + c_m['fp'] + c_m['fn'])
    
    results['Precision'] = c_m['tp'] / (c_m['tp'] + c_m['fp'])
    
    results['Recall'] = c_m['tp'] / (c_m['tp'] + c_m['fn'])
    
    results['Specificity'] = c_m['tn'] / (c_m['tn'] + c_m['fp'])
    
    results['F1-Score'] = 2 * results['Precision'] * results['Recall'] \
        / (results['Precision'] + results['Recall'])
    
    results['Cost Error'] = loss_history[-1]
    
    results['AUC'] = auc(roc1, roc2)
    
    results['PR-Score'] = average_precision_score(corrects, predictions)
    
    if plot:
        plt.title('ROC Curve')
        plt.plot(roc1, roc2, color='darkorange', 
                 label='ROC curve (area %0.2f)' % results['AUC'])
        plt.plot([0, 1], [0, 1], color='navy', linestyle = '--')
        plt.legend(loc='lower right')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.show()

        pr1, pr2, _ = precision_recall_curve(corrects, predictions)

        plt.title('Precision-Recall Curve')
        plt.step(pr2, pr1, color='black', alpha=0.2,
                 label='PR curve (area %0.2f)' % results['PR-Score'])
        plt.fill_between(pr2, pr1, alpha=0.2, color='black')
        plt.legend(loc='lower right')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.show()
    
    return(results)

### Cross-Validation
The traditional cross-validation technique is here implemented, with some tweakings to exploit the Spark parallelisation.  
* `transform_c_v()` transforms the RDD to allow the cross-validation split: each record is randomly assigned to one fold (which range depends on the **folds** number chosen) by adding a label assigned by the `randrange()` functions;  
* `get_block_data()` takes the original train RDD and splits it in **rdd_train** and **rdd_test** according to the labels of each element and the training fold extracted **fold_train**;

In [10]:
def transform_c_v(rdd, folds):
    random.seed()
    rdd_cv = rdd.map(
        lambda x: (
            x[0],
            x[1],
            randrange(folds)
        )
    )
    return rdd_cv


def get_block_data(rdd, fold_train):
    rdd_train = rdd.filter(
        lambda x:
            x[2] != fold_train
    ).map(
        lambda x: (
            x[0],
            x[1]
        )
    )
    
    rdd_test = rdd.filter(
        lambda x:
            x[2] == fold_train
    ).map(
        lambda x: (
            x[0],
            x[1]
        )
    )
    return rdd_train, rdd_test

The proper cross-validation process is here executed: given an RDD and a number of folds **folds** in which to split the dataset (and the usual `train()` parameters, to pass to that function when called), it iterates the training and testing phases selecting each time the proper **rdd_test** and **rdd_train** split, according to the appropriate training fold **fold** selected.  

The accuracy of the trained model (obtained using the previously seen evaluating functions) is recorded for each fold iteration and then finally averaged in **average_accuracy**.  
The implementation also provides additional information about the time elapsed and the fold or training iteration frequency in the computation: these information can be displayed or not by acting on the **verbose** and **verbose_train** flags (respectively regarding information for the cross-validation process and the specific training iterations).

In [11]:
def cross_validation(rdd, folds=10, iterations=None,
                     learning_rate=None, lambda_reg=None,
                     verbose=True, verbose_train=False):
    if verbose:
        start_time = time.time()
        
    total_accuracy = 0
    rdd_cv = transform_c_v(rdd, folds)

    for fold in range(folds):
        rdd_train, rdd_test = get_block_data(rdd_cv, fold)
        
        if verbose:
            tot_train = rdd_train.count()
            tot_test = rdd_test.count()
            perc_test = tot_test / (tot_test + tot_train) * 100
            print('Fold %2d of %2d' % (fold + 1, folds))
            print('Train %4d\tTest %4d\t|\t%0.1f %% train' %
                  (tot_train, tot_test, perc_test)
            )
        
        b, w, loss_history = train(
            rdd_train,
            iterations=iterations,
            lambda_reg=lambda_reg,
            verbose=verbose_train
        )
        
        results = evaluate(rdd_test, b, w, plot=False)
        accuracy = results['Accuracy']
        total_accuracy += accuracy
        if verbose:
            print('Accuracy: %0.4f\n' % accuracy)

    average_accuracy = total_accuracy / folds
    if verbose:
        total_time = time.time() - start_time
        print('Average accuracy: %0.4f' % average_accuracy)
        print('Total time: %0.2f s\nFold frequency: %0.2f Hz' % 
              (total_time, (folds / total_time))
        )
          
    return average_accuracy

In [12]:
cross_validation(rdd, folds=2, iterations=20, 
                 learning_rate=1, lambda_reg=0.1,
                 verbose_train=True)

Fold  1 of  2
Train 2223	Test 2378	|	51.7 % train
It.    0	|	Loss: 0.6931	|	Time: 0.16 s
It.    4	|	Loss: 0.2785	|	Time: 0.78 s
It.    8	|	Loss: 0.2217	|	Time: 1.56 s
It.   12	|	Loss: 0.2149	|	Time: 2.18 s
It.   16	|	Loss: 0.2126	|	Time: 2.71 s

Total time: 3.22 s
Iters frequency: 6.22 Hz
prec 0.9140
Accuracy: 0.9222

Fold  2 of  2
Train 2378	Test 2223	|	48.3 % train
It.    0	|	Loss: 0.6931	|	Time: 0.17 s
It.    4	|	Loss: 0.2584	|	Time: 0.74 s
It.    8	|	Loss: 0.2154	|	Time: 1.33 s
It.   12	|	Loss: 0.2064	|	Time: 1.84 s
It.   16	|	Loss: 0.2027	|	Time: 2.38 s

Total time: 2.77 s
Iters frequency: 7.22 Hz
prec 0.9268
Accuracy: 0.9249

Average accuracy: 0.9235
Total time: 7.60 s
Fold frequency: 0.26 Hz


0.9235399128387487